In [12]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import History
import time
from google.colab import files
import io

In [13]:
start_time = time.time()

Se establece "low_memory=False", pues hay un problema dtype

In [21]:

#cargar datos con la URL del raw de GitHub
url = 'https://raw.githubusercontent.com/fjguillen96/TFM/main/train.csv'
df = pd.read_csv(url , low_memory=False)



#df_2 = pd.read_csv('test.csv' , low_memory=False)
#Id = df_2['Id']
#percent = df_2.shape[0]/(df_1.shape[0]+df_2.shape[0])
#df_1 = df_1.drop(['Customers'],axis = 1)
#df_2 = df_2.drop(['Id'],axis = 1)
#df = pd.concat([df_1,df_2], axis = 0)
#print(df_1.shape[0])
#print(round(percent,5))
#print((1-round(percent,7)) * df.shape[0])

In [ ]:
store = pd.read_csv('store.csv')
print(store.shape)
store.head(5)

"on = 'Store'" establece que se buscarán los valores comunes en dicha columna, y no en parejas o tríos de características iguales.
"how='inner'" establece que se buscarán valores comunes.

In [ ]:
df_new = df.merge(store,on = ['Store'], how='inner')
print(df_new.shape)

In [ ]:
print("Distinct number of Stores :", len(df_new["Store"].unique()))
print("Distinct number of Days :", len(df_new["Date"].unique()))
print("Average daily sales of all stores : ",round(df_new["Sales"].mean(),2))

## Extraemos los datos temporales de las fechas

In [ ]:
df_new['Date'] = pd.to_datetime(df_new['Date'], infer_datetime_format=True)
df_new["Month"] = df_new["Date"].dt.month
df_new["Quarter"] = df_new["Date"].dt.quarter
df_new["Year"] = df_new["Date"].dt.year
df_new["Day"] = df_new["Date"].dt.day
df_new["Week"] = df_new["Date"].dt.isocalendar().week

df_new["Season"] = np.where(df_new["Month"].isin([3,4,5]),"Spring",
np.where(df_new["Month"].isin([6,7,8]),"Summer",
np.where(df_new["Month"].isin([9,10,11]),"Fall",
np.where(df_new["Month"].isin([12,1,2]),"Winter","None"))))

print(df_new[["Date","Year","Month","Day","Week","Quarter","Season"]].head())

## Predicción

In [ ]:
plt.figure(figsize=(15,8))
plt.hist(df_new["Sales"])
plt.title("Histogram for Store Sales")
plt.xlabel("bins")
plt.xlabel("Frequency")
plt.show()

Sacamos los nulos

In [ ]:
df_new.isnull().sum()

Lo pasamos a porcentaje para visualizarlo mejor

In [ ]:
df_new.isnull().sum()/df_new.shape[0] * 100

In [ ]:
df_new["CompetitionDistance"]=df_new["CompetitionDistance"].fillna(df_new["CompetitionDistance"].mode()[0])
df_new["CompetitionDistance"].isnull().sum()/df_new.shape[0] * 100

## Más visualización para entender los datos

In [ ]:
sns.set(style="whitegrid")
ax = sns.barplot(x="Season", y="Sales", data=df_new)

In [ ]:
ax = sns.barplot(x="Assortment", y="Sales", data=df_new)

In [ ]:
ax = sns.barplot(x="StoreType", y="Sales", data=df_new)

In [ ]:
ax = sns.barplot(x="Season", y="Sales", data=df_new,estimator=np.size)

In [ ]:
ax = sns.barplot(x="Assortment", y="Sales", data=df_new,estimator=np.size)

In [ ]:
ax = sns.barplot(x="StoreType", y="Sales", data=df_new,estimator=np.size)

## Pasamos a variable dummy de una forma distinta (Manual)

In [ ]:
target = ["Sales"]
numeric_columns = ["Customers","Open","Promo","Promo2",
"StateHoliday","SchoolHoliday","CompetitionDistance"]
categorical_columns = ["DayOfWeek","Quarter","Month","Year",
"StoreType","Assortment","Season"]

In [ ]:
def create_ohe(df, col):
    
    le = LabelEncoder()
    a=le.fit_transform(df[col]).reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    column_names = [col+ "_"+ str(i) for i in le.classes_]

    return(pd.DataFrame(ohe.fit_transform(a),columns =column_names))

In [ ]:
temp = df_new[numeric_columns]
for column in categorical_columns:
    temp_df = create_ohe(df_new,column)
    temp = pd.concat([temp,temp_df],axis=1)

In [ ]:
print("Shape of Data:",temp.shape)
print("Distinct Datatypes:",temp.dtypes.unique())

In [ ]:
temp["StateHoliday"]= np.where(temp["StateHoliday"]== '0',0,1)
temp.dtypes.unique()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(temp,df_new[target],test_size = 0.2,random_state=2018)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=2018)

## Datos del dataset

In [ ]:
mean_sales = y_train.mean()
print("Average Sales :",mean_sales)
print("MAE for Test Data:",abs(y_test - mean_sales).mean()[0])

## NN

### Rule 1: Start with small architectures

In the case of DNNs, it is always advised to
start with a single-layer network with around
100–300 neurons. Train the network and measure
performance using the defined metrics (while
defining the baseline score). If the results are not
encouraging, try adding one more layer with the
same number of neurons and repeating the process.

### Rule 2: When small architectures (with two layers) fail, increase the size

When the results from small networks are not
great, you need to increase the number of neurons
in layers by three to five times (i.e., around 1,000
neurons in each layer). Also, increase regularization to 0.3, 0.4,
or 0.5 for both layers and repeat the process for
training and performance measurement.

### Rule 3: When larger networks with two layers fail, go deeper

Try increasing the depth of the network with more
and more layers while maintaining regularization with
dropout layers (if required) after each dense (or your
selected layer) with a dropout rate between 0.2 and 0.5.

### Rule 4: When larger and deeper networks also fail, go even larger and even deeper

In case large networks with ~1000 neurons and five
or six layers also don’t give the desired performance,
try increasing the width and depth of the network.
Try adding layers with 8,000–10,000 neurons per
layer and a dropout of 0.6 to 0.8.

### Rule 5: When everything fails, revisit the data

If all the aforementioned rules fail, revisit the
data for improved feature engineering and
normalization, and then you will need to try other
ML alternatives.

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim = 44,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_absolute_error",metrics=["mean_absolute_error"])
model.fit(x_train.values,y_train.values, validation_data=(x_val,y_val),epochs=10,batch_size=64)

In [ ]:
result = model.evaluate(x_test.values,y_test.values)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

### Improve Model (add layers)

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim = 44,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])

history = model.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=10,batch_size=64)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

### More Improve Model (add epochs)

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim = 44,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=15,batch_size=64).result = model.evaluate(x_test,y_test)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

### More More Improve Model (add neurons)

In [ ]:
model = Sequential()
model.add(Dense(350,input_dim = 44,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=15,batch_size=64)

result = model.evaluate(x_test,y_test)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",
    str(round(result[i],2)))

#### Let us now try deeper models for the same architecture. Additionally, we add a new optional configuration to the model to record the history of various metrics during the training process. This can be done by adding the callbacks parameter

In [ ]:
history = History()

model = Sequential()
model.add(Dense(350,input_dim = 44,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=15,batch_size=64,callbacks=[history])

result = model.evaluate(x_test,y_test)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

### Plot

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model's Training & Validation loss across epochs")
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

### Checking

In [ ]:
#Aquí se añade una columna, ['nombre'], por tanto ahora se tiene las columnas Sales (de antes) y Prediction
y_test["Prediction"] = model.predict(x_test)

#Aquí se hace un rename de las existentes, .columns, por tanto ahora se tiene las columnas Actual Sales y Predicted Sales
y_test.columns = ["Actual Sales","Predicted Sales"]
print(y_test.head(10))

print("MSE :",mean_squared_error(y_test["Actual Sales"].values,y_test["Predicted Sales"].values))
print("MAE :",mean_absolute_error(y_test["Actual Sales"].values,y_test["Predicted Sales"].values))

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))